In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load the M2M100 model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

df = pd.read_csv('/kaggle/input/mansi-mono/mansi.csv', index_col=0)

In [3]:
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [4]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [5]:
train_dataset

Dataset({
    features: ['0', '__index_level_0__'],
    num_rows: 146499
})

In [20]:
tokenizer.add_special_tokens({'mask_token': '[MASK]'})
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(128105, 1024, padding_idx=1)

In [7]:
def preprocess_function(examples):
    inputs = tokenizer(examples['0'], return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=128)
    return inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/146499 [00:00<?, ? examples/s]

Map:   0%|          | 0/18312 [00:00<?, ? examples/s]

Map:   0%|          | 0/18313 [00:00<?, ? examples/s]

In [8]:
train_dataset = train_dataset.remove_columns(['__index_level_0__', '0'])
val_dataset = val_dataset.remove_columns(['__index_level_0__', '0'])
test_dataset = test_dataset.remove_columns(['__index_level_0__', '0'])

In [21]:
# Data collator for MLM - this automatically masks some tokens in the input
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True,  # Set to True to enable masked language modeling
    mlm_probability=0.2  # Mask 20% of tokens
)

In [25]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./m2m_smugri_finetuned",
    evaluation_strategy="no", 
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2, 
    weight_decay=0.01,
    save_strategy="epoch", 
    num_train_epochs=1,
#     max_steps=41202,
    logging_dir="./logs",
    report_to="none", 
    fp16=True, 
    load_best_model_at_end=False,                    
)

# Trainer setup
trainer = Trainer(
    model=model,                         # The M2M100 model
    args=training_args,                  # Training arguments
    data_collator=data_collator,         # Data collator for MLM
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
500,1.307600
1000,1.625800
1500,1.477300
2000,1.443000
2500,1.348700
3000,1.306100
3500,1.242100
4000,1.225900
4500,1.174400
5000,1.152300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


TrainOutput(global_step=9156, training_loss=1.2555272346919881, metrics={'train_runtime': 10388.5692, 'train_samples_per_second': 14.102, 'train_steps_per_second': 0.881, 'total_flos': 3.968401947387494e+16, 'train_loss': 1.2555272346919881, 'epoch': 0.9999453939824169})

In [26]:
import shutil
shutil.make_archive("m2m_mlm", 'zip', "/kaggle/working/m2m_smugri_finetuned/checkpoint-9156")

'/kaggle/working/m2m_mlm.zip'

In [27]:
from IPython.display import FileLink

FileLink('m2m_mlm.zip')

/kaggle/working/m2m_mlm.zip